In [1]:
#| default_exp components/details_json


In [2]:
#| export
from __future__ import annotations


# DetailsJSON

> FastHTML component that displays a JSON object in a `details` element.


# Prologue

In [3]:
#| export
from typing import Any
from typing import Literal
from typing import Mapping

from fasthtml.xtend import Style
from olio.common import val_at


In [4]:
#| export

from bridget.bridget import _n
from bridget.bridget import bridge_cfg
from bridget.bridget import Bridget
from bridget.routing import ar
from bridget.routing import RouteProvider


In [5]:
#| export

from fasthtml.components import Details, Summary, Ul, Li, Span


In [6]:
import json
from functools import partial
from pathlib import Path
from typing import cast

from bridget.bridget import bridget_scripts
from bridget.bridget import get_app
from fastcore.test import *
from fastcore.xml import FT
from fastcore.xml import to_xml
from fasthtml.common import show
from rich.console import Console


----


In [7]:
cprint = (console := Console(width=120)).print


----

# Derivation

In [8]:
app, brt, rt = get_app(True, auto_show=True)


Bridget(libraries={'htmx': 'https://unpkg.com/htmx.org@next/dist/htmx.js', 'fasthtml_js': 'https://cdn.jsdeliv…

In [9]:
%%HTML
<style>
    me ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; }
</style>
<details open>
<summary>Apollo astronauts</summary>
<ul>
  <li><span>1</span>: Neil Armstrong</li>
  <li><span>2</span>: Alan Bean</li>
  <li><details>
<summary>Apollo 11</summary>
<ul>
  <li><span>1</span>: Neil Armstrong</li>
  <li><span>2</span>: Alan Bean</li>
  <li><div><span>3</span>: Buzz Aldrin</div></li>
  <li><span>4</span>: Edgar Mitchell</li>
  <li><span>5</span>: Alan Shepard</li>
</ul></li>
  <li><span>4</span>: Edgar Mitchell</li>
  <li><span>5</span>: Alan Shepard</li>
</ul>

</details>


In [10]:
class DetailsJSON(dict):
    def __init__(self, *args, summary:str='', open:bool=False, lvl:int=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.summary, self.open, self.lvl = summary, open, lvl
    def __ft__(self):
        return (
            Style(self._css_) if self.lvl == 0 else (), 
            Details(open=self.open)(
                Summary(self.summary or 'summary', _n),
                Ul()(*(
                    Li(v) if not isinstance(v, Mapping) else 
                    DetailsJSON(v, summary=k, lvl=self.lvl+1) 
                    for k,v in self.items()))))
    _css_ = 'me ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } '


dtl = DetailsJSON({
    '1': 'Neil Armstrong',
    '2': 'Alan Bean',
    '3': 'Buzz Aldrin',
    'letters': {
        'a': 1, 
        'b': 2
        },
    '5': 'Edgar Mitchell',
    '6': 'Alan Shepard'
}, summary='Apollo astronauts', open=True)

test_eq(val_at(dtl, '5'), 'Edgar Mitchell')
test_eq(val_at(dtl, 'letters.a'), 1)
cprint(to_xml(dtl))
show(dtl)


<style>me ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } </style>
<details open><summary>Apollo astronauts
</summary>  <ul>
    <li>Neil Armstrong</li>
    <li>Alan Bean</li>
    <li>Buzz Aldrin</li>
<details><summary>letters
</summary>      <ul>
        <li>1</li>
        <li>2</li>
      </ul>
</details>    <li>Edgar Mitchell</li>
    <li>Alan Shepard</li>
  </ul>
</details>

In [11]:
def walk(m:Mapping, p:str=''):
    for k,v in m.items():
        if isinstance(v, Mapping): yield from walk(v, f"{p}.{k}" if p else k)
        else: yield f"{p}.{k}" if p else k,v 

list(walk(dtl))

[('1', 'Neil Armstrong'),
 ('2', 'Alan Bean'),
 ('3', 'Buzz Aldrin'),
 ('letters.a', 1),
 ('letters.b', 2),
 ('5', 'Edgar Mitchell'),
 ('6', 'Alan Shepard')]

In [12]:
def walk(mapping: Mapping, prefix: str=''):
    stack = [(prefix, list(mapping.items()))]
    while stack:
        p, items = stack[-1]
        if not items: stack.pop(); continue
        k, v = items.pop(0)
        if isinstance(v, Mapping): stack.append((f"{p}.{k}" if p else k, list(v.items())))
        else: yield f"{p}.{k}" if p else k, v

# Test
list(walk(dtl))

[('1', 'Neil Armstrong'),
 ('2', 'Alan Bean'),
 ('3', 'Buzz Aldrin'),
 ('letters.a', 1),
 ('letters.b', 2),
 ('5', 'Edgar Mitchell'),
 ('6', 'Alan Shepard')]

In [13]:
def update(d, other:Mapping|None=None, **kwargs):
    if isinstance(d, dict): d.update(other or {}, **kwargs)
    else:
        for k,v in {**(other or {}), **kwargs}.items(): 
            try: setattr(d, k, v)
            except AttributeError: pass
    return d


In [14]:
d = {'a': 1, 'b':2}
test_eq(update(d, c=3), {'a': 1, 'b':2, 'c':3})
test_eq(update(d, {'b': 22}), {'a': 1, 'b':22, 'c':3})


# DetailsJSON

In [15]:
#| export

def Val(v): 
    c = (
        'null' if v is None else 
        'true' if v is True else 
        'false' if v is False else 
        'string' if isinstance(v, str) else 
        'number' if isinstance(v, (int, float)) else 
        '')
    return Span(v if v is not None else 'None', cls=f"v {c}")
def NameVal(k, v): return Span(Span(k, cls='n'), ': ', Val(v))

class DetailsJSON(RouteProvider):

    def __init__(self, o:Mapping[str, Any], summary:str='', open:bool|Literal['all']=True):
        self.o, self.summary, self.open, self._mounted = o, summary or 'summary', open, False

    def _ipython_display_(self):
        brt = Bridget()
        if bridge_cfg.auto_mount and not self._mounted: brt.mount(self, show=False)
        brt(self)

    @ar('/{dp:path}', methods='get', name='get')
    def __call__(self, dp:str='', all:bool=False): 
        try: d = val_at(self.o, dp, sep='/') if dp else self.o
        except Exception: return None
        return self.__ft__(dp, d, all)
    
    def __ft__(self, dp:str|None=None, d:Mapping|None=None, openall:bool=False):
        if not dp: dp, d, openall = '', self.o, self.open == 'all'
        its = Ul()(*(
            Li()(
                NameVal(k, v) if not isinstance(v, Mapping) else 
                self.__ft__(f"{dp}/{k}", v, openall) if openall else
                Details(hx_get=f"/{self.ar.to()}/"+(f"{dp}/" if dp else '')+f"{k}")(_n,Summary(k),_n)
            ) for k,v in (d or {}).items()))
        return (
            Details(open=True)(Summary(dp.split('/')[-1]), _n, its) if dp else 
            Details(open=self.open is not False, hx_swap='outerHTML')(_n, Style(self._css_), Summary(self.summary), _n, its))

    _css_ = (
        'me ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } '
        '''me .string { color: #24837b; } me .string::before { content: "'"; } me .string::after { content: "'"; } '''
        'me .number { color: #ad8301; } '
        'me .true { color: blue; } '
        'me .false { color: red; } '
        'me .null { color: gray; } '
    )


In [16]:
brt.app.routes.clear()
bridge_cfg.auto_show = False

dtl = DetailsJSON({
    '1': 'Neil Armstrong', '2': 'Alan Bean', '3': 'Buzz Aldrin',
    'letters': { 'a': 1, 'b': True, 'c': {'d': None} },
    '5': 'Edgar Mitchell', '6': 'Alan Shepard'
}, summary='Apollo astronauts')

brt.mount(dtl, '/details', 'details', show=False)

test_eq(dtl.ar.name(), 'details')
test_eq(app.url_path_for('details:get', dp=''), '/details/')
test_eq(app.url_path_for('details:get', dp='letters'), '/details/letters')

dtl()


```html
<details open hx-swap="outerHTML">
  <style>me ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } me .string { color: #24837b; } me .string::before { content: "'"; } me .string::after { content: "'"; } me .number { color: #ad8301; } me .true { color: blue; } me .false { color: red; } me .null { color: gray; } </style>
<summary>Apollo astronauts</summary>
  <ul>
    <li>
<span><span class="n">1</span>: <span class="v string">Neil Armstrong</span></span>    </li>
    <li>
<span><span class="n">2</span>: <span class="v string">Alan Bean</span></span>    </li>
    <li>
<span><span class="n">3</span>: <span class="v string">Buzz Aldrin</span></span>    </li>
    <li>
<details hx-get="/details/letters">
<summary>letters</summary>
</details>    </li>
    <li>
<span><span class="n">5</span>: <span class="v string">Edgar Mitchell</span></span>    </li>
    <li>
<span><span class="n">6</span>: <span class="v string">Alan Shepard</span></span>    </li>
  </ul>
</details>
```

In [17]:
dtl('letters/c')


```html
<details open><summary>c</summary>
  <ul>
    <li>
<span><span class="n">d</span>: <span class="v null">None</span></span>    </li>
  </ul>
</details>
```

In [18]:
# cprint(brt.cli.get('/details/letters', headers={'hx-request': '1'}).text)
cprint(brt.cli.get('/details/letters/c', headers={'hx-request': '1'}).text)


<details open><summary>c</summary>
   <ul>
     <li>
<span><span class="n">d</span>: <span class="v null">None</span></span>     </li>
   </ul>
</details>

In [19]:
bridge_cfg.auto_show = True
dtl()

details(('\n', style(('me ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } me .string { color: #24837b; } me .string::before { content: "\'"; } me .string::after { content: "\'"; } me .number { color: #ad8301; } me .true { color: blue; } me .false { color: red; } me .null { color: gray; } ',),{}), summary(('Apollo astronauts',),{}), '\n', ul((li((span((span(('1',),{'class': 'n'}), ': ', span(('Neil Armstrong',),{'class': 'v string'})),{}),),{}), li((span((span(('2',),{'class': 'n'}), ': ', span(('Alan Bean',),{'class': 'v string'})),{}),),{}), li((span((span(('3',),{'class': 'n'}), ': ', span(('Buzz Aldrin',),{'class': 'v string'})),{}),),{}), li((details(('\n', summary(('letters',),{}), '\n'),{'hx-get': '/details/letters'}),),{}), li((span((span(('5',),{'class': 'n'}), ': ', span(('Edgar Mitchell',),{'class': 'v string'})),{}),),{}), li((span((span(('6',),{'class': 'n'}), ': ', span(('Alan Shepard',),{'class': 'v string'})),{}),),{})),{})),{'open': True, 'hx-swap': 'outerHTML'})

In [20]:
brt('/details/');

In [21]:
bridge_cfg.auto_mount = True

apollo_astronauts = json.load(Path('static/apollo_astronauts.json').open())

(astro := DetailsJSON(apollo_astronauts, summary='Apollo astronauts'))


In [22]:
request = {
    'headers': {
        'HX-Request': 'true',
        'HX-Current-URL': 'vscode-webview://1ql27...enderer'
    },
    'headerNames': {
        'hx-request': 'HX-Request',
        'hx-current-url': 'HX-Current-URL'
    },
    'status': 0,
    'method': 'GET',
    'url': '/DetailsJSON_5096628128/Apollo 11/Buzz Aldrin',
    'async': True,
    'timeout': 0,
    'withCredentials': False,
    'body': None,
    'req_id': '68ffadb0-958d-4346-b314-d9d62ca247d7'
}

response = {
    'headers': {
        'content-length': '441',
        'content-type': 'text/html; charset=utf-8',
        'last-modified': 'Fri, 15 Nov 2024 16:22:15 GMT',
        'cache-control': 'no-store, no-cache, must-revalidate'
    },
    'status': 200,
    'statusText': 'OK',
    'data': '<details open><summary>Buzz Aldrin</summary>\n   <ul>\n     '
'<li>Pilot on Gemini 12 and Lunar Module pilot on Apollo 11.</li>\n     '
'<li>Aldrin was the second person to walk on the moon.</li>\n     <li>The maiden '
'name of Aldrin&#x27;s mother was &quot;Moon.&quot;</li>\n     <li>While Neil was '
'the first human to step onto the moon, I&#x27;m the first alien from another '
'world to enter a spacecraft that was going to Earth.</li>\n   '
'</ul>\n</details>',
    'xml': None,
    'finalUrl': 'http://nb/DetailsJSON_5096628128/Apollo%2011/Buzz%20Aldrin',
    'req_id': '68ffadb0-958d-4346-b314-d9d62ca247d7'
}

In [23]:
(req := DetailsJSON(request, summary='request', open='all'))


In [24]:
(resp := DetailsJSON(response, summary='response'))


# Colophon
----


In [25]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '40_details_json.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
